In [1]:
import setVCD

# setVCD Motivating Example
## Scenario:
Here is an excerpt of the VCD file we use for testing (under `tests/fixtures/wave.vcd`):

![alt text](img/gtkwave.png "Title")

You are debugging an input stream (`io_input`), and would like to see the data only when the transaction is valid. This notebook goes through the steps of doing this with `setVCD`.

## Creating the object
You must specifify the path and name of the clock signal in the file.

In [2]:
# Load VCD file
vcd_path = "./tests/fixtures/wave.vcd"
sv = setVCD.SetVCD(vcd_path, clock="TOP.clk")

## Getting and filtering signals
The primary goal here is to find the timesteps that we care about using the condition function and set operations.

1. Get sets of timesteps with the `SetVCD.get(signal, condition)`, where signal is the signal name (`str`) and condition is a function that takes in a signal value and returns a boolean.
   - For example, to get rising edges we use $\texttt{rising\_edges} = (\texttt{clk}(t-1) = 0) \land (\texttt{clk}(t) = 1)$
3. Use set operations to extract only the timesteps we care about.
   - For example, to get timesteps where there is a rising edge and the reset signal is LOW: $\texttt{clock\_updates} = \texttt{rising\_edges} \cap \texttt{reset\_is\_0}$
   - Note that in Python the set intersection operator ($\cap$) is `&`
5. Use the `SetVCD.get_value(signal, timesteps)` to retrieve the bits at those timesteps of interest. 

In [3]:
rising_edges = sv.get("TOP.clk", lambda sm1, s: sm1 == 0 and s == 1)
print(f"1: Get timesteps with a rising edge : \n{str(sorted(rising_edges)[0:10])[:-1]}, ...]")

1: Get timesteps with a rising edge : 
[34, 36, 38, 40, 42, 44, 46, 48, 50, 52, ...]


We can search the VCD signals for the inputs to our accelerator. Since we are searching for input signals, we can use the following query to find the rest of our signal sof interest.

In [4]:
sv.search("Accelerator.io_input")

['TOP.Accelerator.io_input_valid',
 'TOP.Accelerator.io_input_ready',
 'TOP.Accelerator.io_input_payload_last',
 'TOP.Accelerator.io_input_payload_fragment_value_0[15:0]']

In [5]:
# Get timesteps where reset is 0
reset_is_0 = sv.get("TOP.reset", lambda s: s == 0)

# Get times when input_valid and output_ready are asserted.
in_valid = sv.get("TOP.Accelerator.io_input_valid", lambda s: s == 1)
in_ready = sv.get("TOP.Accelerator.io_input_ready", lambda s: s == 1)

# Get timesteps of valid inputs
valid_input_timesteps = rising_edges & reset_is_0 & in_ready & in_valid
print(f"2: Reduce to timesteps only when the input has a transaction: \n{str(sorted(valid_input_timesteps)[0:10])[:-1]}, ...]")

2: Reduce to timesteps only when the input has a transaction: 
[42, 52, 62, 72, 82, 98, 118, 128, 148, 158, ...]


Once We've acquired the valid timesteps, we can get the values from the data signal (`payload_fragment_value`). 

The inputs to the Accelerator module are **fixed point SQ1.15**, so we specify this in the `value_type`.

In [6]:
inputs = sv.get_values(
    "TOP.Accelerator.io_input_payload_fragment_value_0[15:0]",
    valid_input_timesteps,
    value_type=setVCD.FP(frac = 15, signed = True)
)
print(f"3: Get the values at those filtered timesteps: \n{str(inputs[0:10])[:-1]}, ...]")

3: Get the values at those filtered timesteps: 
[0.5999755859375, 0.5999755859375, 0.5999755859375, 0.5999755859375, 0.5999755859375, 0.5999755859375, 0.5999755859375, 0.5999755859375, 0.5999755859375, 0.5999755859375, ...]


## Evaluating 
We've filtered down the `input_fragment_value` signal only to timesteps that we care about! We can now create various assertions and inspections on the signal which can significantly improve debugging over simply looking at VCD waves.

In [7]:
# Get (timestep, value) tuples instead of just output
input_with_t = sv.get_values_with_t(
    "TOP.Accelerator.io_input_payload_fragment_value_0[15:0]",
    valid_input_timesteps,
    value_type=setVCD.FP(frac = 15, signed = True)
)

print(f"Received expected (100) inputs? {len(inputs) == 100}")

avg_latency = 0
avg_value = 0
for i in range(1, len(input_with_t)):
    t, v = input_with_t[i]
    tm1, vm1 = input_with_t[i-1]
    avg_latency = (avg_latency + (t - tm1)) / 2
    avg_value = (avg_value + v) / 2

print(f"Average clocks between inputs: {avg_latency:.2f}")
print(f"Average input value: {avg_value:.5f}")

Received expected (100) inputs? True
Average clocks between inputs: 10.38
Average input value: 0.59998
